In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

pd.options.mode.chained_assignment = None

In [2]:
# Import data
df = pd.read_csv("ripa_stops_datasd.csv", low_memory=False)

In [3]:
df.head()

,stop_id,ori,agency,exp_years,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,...,beat_name,pid,isstudent,perceived_limited_english,perceived_age,perceived_gender,gender_nonconforming,gend,gend_nc,perceived_lgbt
0,2443,CA0371100,SD,10,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",...,Pacific Beach 122,1,0,0,25,Male,0,1,NaN,No
1,2444,CA0371100,SD,18,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",...,Mission Beach 121,1,0,0,25,Male,0,1,NaN,No
2,2447,CA0371100,SD,1,2018-07-01,00:05:43,15,1,10,Other,...,El Cerrito 822,1,0,0,30,Male,0,1,NaN,No
3,2447,CA0371100,SD,1,2018-07-01,00:05:43,15,1,10,Other,...,El Cerrito 822,2,0,0,30,Female,0,2,NaN,No
4,2448,CA0371100,SD,3,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",...,Ocean Beach 614,1,0,0,23,Male,0,1,NaN,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464044 entries, 0 to 464043
Data columns (total 29 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   stop_id                    464044 non-null  int64  
 1   ori                        464044 non-null  object 
 2   agency                     464044 non-null  object 
 3   exp_years                  464044 non-null  int64  
 4   date_stop                  464044 non-null  object 
 5   time_stop                  464044 non-null  object 
 6   stopduration               464044 non-null  int64  
 7   stop_in_response_to_cfs    464044 non-null  int64  
 8   officer_assignment_key     464044 non-null  int64  
 9   assignment                 464044 non-null  object 
 10  intersection               43743 non-null   object 
 11  address_block              417070 non-null  float64
 12  land_mark                  56 non-null      object 
 13  address_street             44

## Getting columns we're interested in

In [5]:
# Get reduced dataframe with relevant info
rdf= df[['stop_id',
         'beat',
         'isschool',
         'date_stop', 
         'time_stop', 
         'stopduration',
         'stop_in_response_to_cfs', # cfs == call for service
         'officer_assignment_key',
         'assignment',
         'exp_years', 
         'pid',
         'isstudent',
         'perceived_limited_english',
         'perceived_age',
         'perceived_gender']]

In [6]:
rdf.head()

,stop_id,beat,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,exp_years,pid,isstudent,perceived_limited_english,perceived_age,perceived_gender
0,2443,122,0,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",10,1,0,0,25,Male
1,2444,121,0,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",18,1,0,0,25,Male
2,2447,822,0,2018-07-01,00:05:43,15,1,10,Other,1,1,0,0,30,Male
3,2447,822,0,2018-07-01,00:05:43,15,1,10,Other,1,2,0,0,30,Female
4,2448,614,0,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",3,1,0,0,23,Male


In [7]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464044 entries, 0 to 464043
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   stop_id                    464044 non-null  int64 
 1   beat                       464044 non-null  int64 
 2   isschool                   464044 non-null  int64 
 3   date_stop                  464044 non-null  object
 4   time_stop                  464044 non-null  object
 5   stopduration               464044 non-null  int64 
 6   stop_in_response_to_cfs    464044 non-null  int64 
 7   officer_assignment_key     464044 non-null  int64 
 8   assignment                 464044 non-null  object
 9   exp_years                  464044 non-null  int64 
 10  pid                        464044 non-null  int64 
 11  isstudent                  464044 non-null  int64 
 12  perceived_limited_english  464044 non-null  int64 
 13  perceived_age              464044 non-null  

## Perceived Gender

In [8]:
rdf['perceived_gender'].unique()

array(['Male', 'Female', 'Transgender woman/girl', nan,
       'Transgender man/boy'], dtype=object)

In [9]:
# Put nonbinary gender values into one bin
rdf['perceived_gender'].replace(['Transgender woman/girl', 'Transgender man/boy', np.nan], 'Other', inplace=True)

In [10]:
# Prepare for one hot encoding
rdf['perceived_gender'] = rdf['perceived_gender'].apply(lambda x: 'gender_' + x.lower())

In [11]:
# Use one hot encoder to fit and transform perceived_gender column
g_encoder =  OneHotEncoder().fit(rdf[['perceived_gender']])
g_transformed = g_encoder.transform(rdf[['perceived_gender']]).toarray()
g_encoder.categories_

[array(['gender_female', 'gender_male', 'gender_other'], dtype=object)]

In [12]:
# Attach each binary column to the dataframe
for index, category in enumerate(np.concatenate(g_encoder.categories_)):
    rdf[category] = g_transformed[:,index]

In [13]:
rdf = rdf.drop(['perceived_gender'], axis=1)

In [14]:
rdf.head()

,stop_id,beat,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,exp_years,pid,isstudent,perceived_limited_english,perceived_age,gender_female,gender_male,gender_other
0,2443,122,0,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",10,1,0,0,25,0.0,1.0,0.0
1,2444,121,0,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",18,1,0,0,25,0.0,1.0,0.0
2,2447,822,0,2018-07-01,00:05:43,15,1,10,Other,1,1,0,0,30,0.0,1.0,0.0
3,2447,822,0,2018-07-01,00:05:43,15,1,10,Other,1,2,0,0,30,1.0,0.0,0.0
4,2448,614,0,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",3,1,0,0,23,0.0,1.0,0.0


# Assignment

In [15]:
rdf['assignment'].unique()

array(['Patrol, traffic enforcement, field operations', 'Other',
       'Gang enforcement', 'Roadblock or DUI sobriety checkpoint',
       'Investigative/detective', 'Special events', 'Task force',
       'Narcotics/vice',
       'K1-12 public school inlcuding school resource officer or school police officer',
       'Compliance check'], dtype=object)

In [16]:
# Prepare assignment dictionary for one hot encoding
assign_dict = {'Patrol, traffic enforcement, field operations': 'assignment_A',
               'Gang enforcement': 'assignment_B',
               'Compliance check': 'assignment_C',
               'Special events': 'assignment_D',
               'Roadblock or DUI sobriety checkpoint': 'assignment_E',
               'Narcotics/vice': 'assignment_F',
               'Task force': 'assignment_G',
               'K1-12 public school inlcuding school resource officer or school police officer': 'assignment_H',
               'Investigative/detective': 'assignment_I',
               'Other': 'assignment_J'}

rdf['assignment'].replace(assign_dict, inplace=True)

In [17]:
# Use one hot encoder to fit and transform assignment column
a_encoder =  OneHotEncoder().fit(rdf[['assignment']])
a_transformed = a_encoder.transform(rdf[['assignment']]).toarray()
a_encoder.categories_

[array(['assignment_A', 'assignment_B', 'assignment_C', 'assignment_D',
        'assignment_E', 'assignment_F', 'assignment_G', 'assignment_H',
        'assignment_I', 'assignment_J'], dtype=object)]

In [18]:
# Attach transformed columns to dataframe
for index, category in enumerate(np.concatenate(a_encoder.categories_)):
    rdf[category] = a_transformed[:,index]

In [19]:
rdf = rdf.drop(['assignment'], axis=1)

In [20]:
rdf.head()

,stop_id,beat,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,pid,...,assignment_A,assignment_B,assignment_C,assignment_D,assignment_E,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J
0,2443,122,0,2018-07-01,00:01:37,30,0,1,10,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2444,121,0,2018-07-01,00:03:34,10,0,1,18,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2447,822,0,2018-07-01,00:05:43,15,1,10,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2447,822,0,2018-07-01,00:05:43,15,1,10,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2448,614,0,2018-07-01,00:19:06,5,0,1,3,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Stop time

In [21]:
# Really weird occurrences with ancient years in time_stop column!
weird_ids = rdf['time_stop'].apply(lambda x: (len(x.split()) == 2))
rdf[weird_ids]

,stop_id,beat,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,pid,...,assignment_A,assignment_B,assignment_C,assignment_D,assignment_E,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J
97194,92205,524,0,2019-01-18,1900-01-01 04:48:00,30,1,1,10,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113838,109883,313,0,2019-02-22,1900-01-01 02:53:09,15,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124771,120608,722,0,2019-03-16,1900-01-01 05:47:19,30,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127568,123099,512,0,2019-03-22,1900-01-01 01:56:00,25,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138748,132994,446,0,2019-04-11,1900-01-01 03:33:24,45,1,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438827,397980,723,0,2021-01-29,1899-12-30 00:00:00,180,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443226,402207,124,0,2021-02-10,1899-12-30 00:00:00,10,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443281,402252,836,0,2021-02-10,1899-12-30 00:00:00,90,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
453554,411242,834,0,2021-03-06,1899-12-30 00:00:00,30,0,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Create datetime objects from time_stop values
# (taking into account the weird values)
def make_timestamp(row):
    if len(row.time_stop.split()) == 2:
        row.time_stop = row.time_stop.split()[1]
    
    timestr = row.date_stop + ' ' + row.time_stop
    
    return datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')

In [23]:
# Create timestamp column
rdf['timestamp'] = rdf.apply(lambda row: make_timestamp(row), axis=1)

In [24]:
# Play around with timestamp
min_ts = min(rdf['timestamp'])
max_ts = max(rdf['timestamp'])
print(min_ts)
print(max_ts)
print(max(rdf['timestamp']))
print(max(rdf['timestamp']) - min(rdf['timestamp']))
print((max_ts - min_ts).total_seconds())

2018-07-01 00:01:37
2021-03-31 23:39:35
2021-03-31 23:39:35
1004 days 23:37:58
86830678.0


In [25]:
rdf.head()

,stop_id,beat,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,pid,...,assignment_B,assignment_C,assignment_D,assignment_E,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J,timestamp
0,2443,122,0,2018-07-01,00:01:37,30,0,1,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:01:37
1,2444,121,0,2018-07-01,00:03:34,10,0,1,18,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:03:34
2,2447,822,0,2018-07-01,00:05:43,15,1,10,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43
3,2447,822,0,2018-07-01,00:05:43,15,1,10,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43
4,2448,614,0,2018-07-01,00:19:06,5,0,1,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:19:06


# Day, Month, Year

In [26]:
print(max(rdf['timestamp']).year)
print(max(rdf['timestamp']).month)
print(max(rdf['timestamp']).day)
print(max(rdf['timestamp']).weekday()) # 0 -> 6 == Monday -> Sunday

2021
3
31
2


In [27]:
# Attach separate columns for year, month, day and weekday from timestamps
rdf['year'] = rdf['timestamp'].apply(lambda ts: ts.year)
rdf['month'] = rdf['timestamp'].apply(lambda ts: ts.month)
rdf['day'] = rdf['timestamp'].apply(lambda ts: ts.day)
rdf['weekday'] = rdf['timestamp'].apply(lambda ts: ts.weekday())

In [28]:
rdf.head()

,stop_id,beat,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,pid,...,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J,timestamp,year,month,day,weekday
0,2443,122,0,2018-07-01,00:01:37,30,0,1,10,1,...,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:01:37,2018,7,1,6
1,2444,121,0,2018-07-01,00:03:34,10,0,1,18,1,...,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:03:34,2018,7,1,6
2,2447,822,0,2018-07-01,00:05:43,15,1,10,1,1,...,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43,2018,7,1,6
3,2447,822,0,2018-07-01,00:05:43,15,1,10,1,2,...,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43,2018,7,1,6
4,2448,614,0,2018-07-01,00:19:06,5,0,1,3,1,...,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:19:06,2018,7,1,6
